In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
import datetime
import os
from tqdm import tqdm
import time
from collections import defaultdict


import gensim
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models
from torchvision.models import VGG16_Weights, ResNet152_Weights
import torchvision
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import TargetEncoder


print('Is CUDA available ?', torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

"""
vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)
for param in vgg16.parameters():
    param.requires_grad = False
"""

C:\Users\User\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Is CUDA available ? True


'\nvgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)\nfor param in vgg16.parameters():\n    param.requires_grad = False\n'

In [2]:
validation_rate = 0.1
total_train_json = pd.read_json("train_data.json")
test_json = pd.read_json("test_data.json")


json_data_training_images_total = total_train_json['img_filepath']
train_num = int(json_data_training_images_total.shape[0] * (1 - validation_rate))

#json_data_validation_images = json_data_training_images_total.iloc[train_num: ]
#json_data_training_images = json_data_training_images_total.iloc[: train_num]

json_data_testing_images = test_json['img_filepath']
labels = pd.read_csv("train_label.csv")
labels.set_index('Pid', inplace = True)

total_train_json['label'] = labels['label'].to_list()

#train_labels = labels.iloc[: train_num]
#validation_labels = labels.iloc[train_num: ]

In [3]:
total_train_json

,Pid,Uid,Title,Alltags,Category,Concept,Subcategory,Postdate,img_filepath,label
0,149005,22687@N84,having a drink,life county wild bird water animal closeup fau...,Food,thirsty,Drinks,1426216890,train/22687@N84/149005.jpg,10.07
1,149948,17614@N19,"Foto Agne Sterberg, Destination Hga Kusten, AG...",hav mitt hga kusten blsippor nordingr klippor ...,Travel&Active&Sports,mitt,Baseball,1426557920,train/17614@N19/149948.jpg,6.27
2,151388,17614@N19,"Foto Agne Sterberg, AGMA Forntid & ventyr AB, ...",is sweden sverige hav soluppgng mitt vr hga ku...,Travel&Active&Sports,mitt,Baseball,1427232557,train/17614@N19/151388.jpg,5.46
3,151389,17614@N19,"Foto Agne Sterberg, AGMA Forntid & ventyr AB, ...",is sweden sverige hav soluppgng mitt vr hga ku...,Travel&Active&Sports,mitt,Baseball,1427192316,train/17614@N19/151389.jpg,5.39
4,151390,17614@N19,"Foto Agne Sterberg, AGMA Forntid & ventyr AB, ...",is sweden sverige hav soluppgng mitt vr hga ku...,Travel&Active&Sports,mitt,Baseball,1427234146,train/17614@N19/151390.jpg,5.36
...,...,...,...,...,...,...,...,...,...,...
14995,254905,35679@N92,Summer 2015,garden cows pumpkins courtyard pomegranates sa...,Holiday&Celebrations,pumpkins,Halloween,1444977582,train/35679@N92/254905.jpg,4.32
14996,382457,44863@N15,www.thravesphoto.co.uk,portrait woman white girl beauty female youth ...,Travel&Active&Sports,diet,"Health,Fitness",1452924796,train/44863@N15/382457.jpg,8.83
14997,241322,32899@N73,DSC06507,street bridge people toronto ontario canada ou...,Entertainment,repeat,Music,1443852954,train/32899@N73/241322.jpg,6.44
14998,318826,28588@N39,... and to all a goodnight.,christmas wood white mobile night weihnachten ...,Holiday&Celebrations,ornaments,Christmas,1450780890,train/28588@N39/318826.jpg,9.46


In [4]:
class LoadDataClass(Dataset):
    def __init__(self, dirpath, labels_data, train, transform=None):
        """ 
        Initialize dataset
        dirpath is a pandas object
        labels_data is a pandas dataframe
        """
        self.dirpath = dirpath
        self.labels_data = labels_data
        self.train = train
        self.filepaths = []
        self.labels = []
        self.transform = transform
        
        for fp in self.dirpath:
            self.filepaths.append(fp)
            image = fp.split('/')[2]
            index = int(image[0: image.find('.')])
            if train:
                self.labels.append(self.labels_data.loc[index].values[0])


    def __getitem__(self, index):
        """ Get a sample from the dataset """
                
        fp = self.filepaths[index]
        image = Image.open(fp)
        image = image.convert('RGB')

        if self.train:
            label = self.labels[index]
        else:
            label = 'Test data'

        if self.transform is not None:
            image = self.transform(image)
        image.unsqueeze(0)
        
        return image, label
    
    def __len__(self):
        """ Get total number of samples in the dataset """
        return len(self.filepaths)

In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

"""
trainset = LoadDataClass(
    dirpath = json_data_training_images,
    labels_data = train_labels,
    train = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation((-30, 30), expand=False),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
)

validationset = LoadDataClass(
    dirpath = json_data_validation_images,
    labels_data = validation_labels,
    train = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
)
"""

testset = LoadDataClass(
    dirpath = json_data_testing_images,
    labels_data = np.zeros(json_data_testing_images.shape[0]),
    train = False,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
)


trainset_total = LoadDataClass(
    dirpath = json_data_training_images_total,
    labels_data = labels,
    train = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation((-30, 30), expand=False),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
)

In [6]:
dataloader = {
    'test': DataLoader(testset, shuffle = False),
    'total_train': DataLoader(trainset_total, batch_size = 32, shuffle = False)
}

In [7]:
class Pretrained_ResNet152():
    def __init__(self):
        # Load pretrained model
        self.pretrained_model = models.resnet152(weights=ResNet152_Weights.DEFAULT)
        # self.pretrained_model = models.vgg16(pretrained=True)

        self.pretrained_model.eval()
        self.pretrained_model.to(device)
        self.c = {}
        with open("imagenet1000_clsidx_to_labels.txt") as f:
            for line in f:
                (key, val) = line.split(":")
                val = val.replace("'", "")
                #val = val.replace(" ", "")
                self.c[int(key)] = val.split(",")[0][1: ]

    def predict(self, image):
        outputs = self.pretrained_model(image)
        
        s = torch.nn.Softmax(dim = 1)
        result = s(outputs)

        prob, predicted = result.sort(1, descending=True)
        #prob = prob.cpu().detach().data.numpy()
        predicted = predicted.cpu().detach().data.numpy()
        outputs = []
        
        for data in predicted:
            output = []
            for j in range(5):
                output.append(self.c[int(data[j])])
            outputs.append(output)

        return outputs
        

In [8]:
pretrained_resnet152 = Pretrained_ResNet152()
train_predicts = []
test_predicts = []

for (x, y) in tqdm(dataloader['total_train']):
    x, y = x.to(device), y.to(device)

    outputs = pretrained_resnet152.predict(x) 
    train_predicts.extend(outputs)

#for (x, y) in tqdm(dataloader['test']):
    #x = x.to(device)

    #outputs = pretrained_resnet152.predict(x) 
    #test_predicts.extend(outputs)

100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [03:56<00:00,  1.98it/s]


In [9]:
train_predicts

[['goose', 'flamingo', 'black swan', 'pelican', 'drake'],
 ['breakwater', 'seashore', 'stone wall', 'lakeside', 'valley'],
 ['breakwater', 'lakeside', 'seashore', 'beacon', 'promontory'],
 ['megalith', 'canoe', 'paddle', 'lakeside', 'seashore'],
 ['sandbar', 'breakwater', 'seashore', 'lakeside', 'beacon'],
 ['lakeside', 'breakwater', 'promontory', 'beacon', 'seashore'],
 ['lakeside', 'snowmobile', 'ski', 'dogsled', 'sandbar'],
 ['barn', 'park bench', 'birdhouse', 'picket fence', 'worm fence'],
 ['valley', 'stone wall', 'lakeside', 'breakwater', 'seashore'],
 ['breakwater', 'seashore', 'lakeside', 'promontory', 'rock crab'],
 ['sandbar', 'lakeside', 'canoe', 'breakwater', 'paddle'],
 ['fountain', 'breakwater', 'geyser', 'seashore', 'cliff'],
 ['breakwater', 'lakeside', 'beacon', 'pier', 'promontory'],
 ['lakeside', 'breakwater', 'sandbar', 'boathouse', 'park bench'],
 ['breakwater', 'sandbar', 'seashore', 'lakeside', 'beacon'],
 ['sandbar', 'valley', 'seashore', 'lakeside', 'breakwater'

In [9]:
total_train_category = total_train_json['Category'].str.replace("&", " ")
total_train_concept = total_train_json['Concept'].str.replace("&", " ")
total_train_subcategory = total_train_json['Subcategory'].str.replace("&", " ")

test_category = test_json['Category'].str.replace("&", " ")
test_concept = test_json['Concept'].str.replace("&", " ")
test_subcategory = test_json['Subcategory'].str.replace("&", " ")

In [10]:
def get_image_catconsub_similarity(category, concept, subcategory, predicts):
    """
    計算利用 Pretrained_ResNet152 產生的前五個圖像相關字和 category, concept, subcategory 各自的平均 cosine similarity
    """
    bert_pretrained_model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(bert_pretrained_model_name)
    model = BertModel.from_pretrained(bert_pretrained_model_name)
    model = model.to(device)

    cat_avg = []
    con_avg = []
    subcat_avg = []

    for i, p in enumerate(predicts):
        clear_output(wait = True)
        print(f"[{i} / {len(predicts)}]")
        cat = torch.tensor([tokenizer.encode(category.iloc[i], add_special_tokens=True)]).to(device)
        con = torch.tensor([tokenizer.encode(concept.iloc[i], add_special_tokens=True)]).to(device)
        sub = torch.tensor([tokenizer.encode(subcategory.iloc[i], add_special_tokens=True)]).to(device)

        with torch.no_grad():
            cat = model(cat)[1][0].cpu().detach().numpy()
            con = model(con)[1][0].cpu().detach().numpy()
            sub = model(sub)[1][0].cpu().detach().numpy()

        CAT = 0
        CON = 0
        SUB = 0
        for img_words in p:
            img_word = torch.tensor([tokenizer.encode(img_words, add_special_tokens=True)]).to(device)
            with torch.no_grad():
                img_word = model(img_word)[1][0].cpu().detach().numpy()
            CAT += cosine_similarity(cat.reshape(1, -1), img_word.reshape(1, -1))[0][0]
            CON += cosine_similarity(con.reshape(1, -1), img_word.reshape(1, -1))[0][0]
            SUB += cosine_similarity(sub.reshape(1, -1), img_word.reshape(1, -1))[0][0]

        cat_avg.append(CAT / 5)
        con_avg.append(CON / 5)
        subcat_avg.append(SUB / 5)
    
    return cat_avg, con_avg, subcat_avg

In [11]:
total_train_cat_avg, total_train_con_avg, total_train_subcat_avg = get_image_catconsub_similarity(total_train_category, total_train_concept, total_train_subcategory, train_predicts)
test_cat_avg, test_con_avg, test_subcat_avg = get_image_catconsub_similarity(test_category, test_concept, test_subcategory, test_predicts)

[4999 / 5000]


In [47]:
train_features = pd.DataFrame()
test_features = pd.DataFrame()

train_features['imgcat_avg'] = total_train_cat_avg
train_features['imgcon_avg'] = total_train_con_avg
train_features['imgsubcat_avg'] = total_train_subcat_avg

test_features['imgcat_avg'] = test_cat_avg
test_features['imgcon_avg'] = test_con_avg
test_features['imgsubcat_avg'] = test_subcat_avg



## Category Target Encoding

In [48]:
cat_targetencoder = TargetEncoder(smooth="auto")
train_features['cat_encoded'] = cat_targetencoder.fit_transform(total_train_json['Category'].to_numpy().reshape(-1, 1), labels.values.reshape(-1, 1))
test_features['cat_encoded'] = cat_targetencoder.transform(test_json['Category'].to_numpy().reshape(-1, 1))

C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## PostDate Target Encoding

In [49]:
train_time = pd.DataFrame()
train_time['time_q'] = total_train_json['Postdate'].astype(int)
train_time["year"] = train_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).year)
train_time["month"] = train_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).month)
train_time["day"] = train_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).day)
train_time["hour"] = train_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).hour)

test_time = pd.DataFrame()
test_time['time_q'] = test_json['Postdate'].astype(int)
test_time["year"] = test_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).year)
test_time["month"] = test_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).month)
test_time["day"] = test_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).day)
test_time["hour"] = test_time["time_q"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).hour)

In [50]:
year_targetencoder = TargetEncoder(smooth="auto")
month_targetencoder = TargetEncoder(smooth="auto")
day_targetencoder = TargetEncoder(smooth="auto")
hg_targetencoder = TargetEncoder(smooth="auto")

train_features['year_encoded'] = year_targetencoder.fit_transform(train_time['year'].to_numpy().reshape(-1, 1), labels.values.reshape(-1, 1))
train_features['month_encoded'] = year_targetencoder.fit_transform(train_time['month'].to_numpy().reshape(-1, 1), labels.values.reshape(-1, 1))
train_features['day_encoded'] = year_targetencoder.fit_transform(train_time['day'].to_numpy().reshape(-1, 1), labels.values.reshape(-1, 1))
train_features['hour_encoded'] = year_targetencoder.fit_transform(train_time['hour'].to_numpy().reshape(-1, 1), labels.values.reshape(-1, 1))

test_features['year_encoded'] = year_targetencoder.transform(test_time['year'].to_numpy().reshape(-1, 1))
test_features['month_encoded'] = year_targetencoder.transform(test_time['month'].to_numpy().reshape(-1, 1))
test_features['day_encoded'] = year_targetencoder.transform(test_time['day'].to_numpy().reshape(-1, 1))
test_features['hour_encoded'] = year_targetencoder.transform(test_time['hour'].to_numpy().reshape(-1, 1))


C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

## Tag Frequency and Target Encoding

In [63]:
train_tags = total_train_json['Alltags'].str.split(" ")
test_tags = test_json['Alltags'].str.split(" ")

train_tags

0        [life, county, wild, bird, water, animal, clos...
1        [hav, mitt, hga, kusten, blsippor, nordingr, k...
2        [is, sweden, sverige, hav, soluppgng, mitt, vr...
3        [is, sweden, sverige, hav, soluppgng, mitt, vr...
4        [is, sweden, sverige, hav, soluppgng, mitt, vr...
                               ...                        
14995    [garden, cows, pumpkins, courtyard, pomegranat...
14996    [portrait, woman, white, girl, beauty, female,...
14997    [street, bridge, people, toronto, ontario, can...
14998    [christmas, wood, white, mobile, night, weihna...
14999    [cars, austria, tour, event, tuning, treffen, ...
Name: Alltags, Length: 15000, dtype: object

In [60]:
tagsfrequency = defaultdict(int)
tagsaveragelabel = defaultdict(int)

for index, i in enumerate(train_tags):
    for j in i:
        tagsfrequency[j] += 1
        tagsaveragelabel[j] += (total_train_json['label'].iloc[index] - tagsaveragelabel[j]) / tagsfrequency[j]
tagsaveragelabel

defaultdict(int,
            {'life': 6.681930835734869,
             'county': 6.835666666666668,
             'wild': 7.207802197802198,
             'bird': 7.248227848101265,
             'water': 7.490709219858151,
             'animal': 6.975776892430275,
             'closeup': 7.367029702970296,
             'fauna': 6.874,
             'swan': 9.990000000000002,
             'drink': 8.01910994764398,
             'wildlife': 6.605423728813558,
             'drinking': 7.215625,
             'neil': 8.734,
             'kerry': 8.605,
             'co': 6.04,
             'waterfowl': 7.575,
             'ornithology': 7.609999999999999,
             'thirsty': 7.167407407407408,
             'muteswan': 8.39,
             'cygnusolor': 8.39,
             'countykerry': 8.823333333333334,
             'cokerry': 8.823333333333334,
             'neilt': 9.31,
             'tackaberry': 8.7925,
             'northkerry': 10.07,
             'neiltackaberry': 8.7925,
            

In [61]:
train_features['numberoftags'] = [len(i) for i in total_train_json['Alltags'].str.split(" ")]
test_features['numberoftags'] = [len(i) for i in test_json['Alltags'].str.split(" ")]

In [67]:
train_tag_weightedvalues = []
test_tag_weightedvalues = []

for row in train_tags:
    count_tags = 0
    value = 0
    for element in row:
        count_tags += tagsfrequency[element]
        value += (tagsfrequency[element] * tagsaveragelabel[element])
    train_tag_weightedvalues.append(value / count_tags)

for row in test_tags:
    count_tags = 0
    value = 0
    for element in row:
        count_tags += tagsfrequency[element]
        value += (tagsfrequency[element] * tagsaveragelabel[element])
    test_tag_weightedvalues.append(value / count_tags)

In [69]:
train_features['tag_weighted_value'] = train_tag_weightedvalues
test_features['tag_weighted_value'] = test_tag_weightedvalues

In [72]:
train_features.to_csv("train_features.csv", index = False)
test_features.to_csv("test_features.csv", index = False)

## Frequency Encoded of Category, Concept, Subccategory and Uid

In [3]:
train_features = pd.read_csv("train_features.csv")
test_features = pd.read_csv("test_features.csv")
test_features

,imgcat_avg,imgcon_avg,imgsubcat_avg,cat_encoded,year_encoded,month_encoded,day_encoded,hour_encoded,numberoftags,tag_weighted_value
0,0.913102,0.923995,0.893891,6.884810,6.430505,6.299635,6.520974,6.504985,38,7.109640
1,0.913463,0.912536,0.931054,6.884810,6.430505,6.299635,6.520974,6.504985,38,7.109640
2,0.875470,0.804715,0.897655,7.459524,6.430505,6.299635,6.504985,6.504985,17,7.833776
3,0.870164,0.894504,0.625052,6.195947,6.430505,6.299635,6.340702,6.299635,6,8.265682
4,0.895613,0.846188,0.880492,6.195947,6.430505,6.299635,6.340702,6.504985,9,6.986714
...,...,...,...,...,...,...,...,...,...,...
4995,0.797631,0.921698,0.929322,6.742603,6.430505,6.339497,6.493121,6.656396,23,6.918538
4996,0.751963,0.863455,0.751963,6.218673,6.430505,6.493121,6.339497,6.699477,14,7.141085
4997,0.898303,0.935752,0.916846,6.122679,6.430505,6.299635,6.430505,6.827331,9,8.335856
4998,0.937403,0.842122,0.916187,6.220443,6.430505,6.346116,6.433462,6.433462,6,6.886312


In [52]:
useridcount = total_train_json['Uid'].value_counts().to_dict()
train_uid_frequency = []
test_uid_frequency = []

for i in total_train_json['Uid']:
    train_uid_frequency.append(useridcount[i])
    
for i in test_json['Uid']:
    if i not in useridcount:
        print(i, " not in train data dictionary")
        test_uid_frequency.append(total_train_json['Uid'].value_counts().mean())
    else:
        test_uid_frequency.append(useridcount[i])


1@N18  not in train data dictionary
1@N18  not in train data dictionary
1@N18  not in train data dictionary
49469@N54  not in train data dictionary
47720@N96  not in train data dictionary
49917@N20  not in train data dictionary
20139@N49  not in train data dictionary
23325@N6  not in train data dictionary
2891@N78  not in train data dictionary
49469@N54  not in train data dictionary
42227@N19  not in train data dictionary
42227@N19  not in train data dictionary
53875@N66  not in train data dictionary
49917@N20  not in train data dictionary
35558@N54  not in train data dictionary
45431@N9  not in train data dictionary
47720@N96  not in train data dictionary
27706@N81  not in train data dictionary
2891@N78  not in train data dictionary
45431@N9  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
26495@N54  not in train data dictionary
53875@N66  not in train data dictionary
2891@N78  not in train data dictionary
2891@N78  not in t

177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not

177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
177@N18  not in train data dictionary
41122@N96  not in train data dictionary
41122@N96  not in train data dictionary
41122@N96  not in train data dictionary
41122@N96  not in train data dictionary
41122@N96  not in train data dictionary
41122@N96  not in train data dictionary
64371@N34  not in train data dictionary
64371@N34  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
6675@N61  not in train data dictionary
25345@N55  not 

11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
11709@N15  not in train data dictionary
29006@N14  not in train data dictionary
29146@N36  not in train data dictionary
48070@N37  not in train data dictionary
27818@N60  not in train data dictionary
27818@N60  not in train data dictionary
27818@N60  not in train data dictionary
44751@N50  not in train data dictionary
44751@N50  not in train data dictionary
20186@N94  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
64895@N58  not in train data dictionary
34558@N3  not in train data dictionary
34558@N3  not in train data dictionary
34558@N3  not in train data dictionary
345

14913@N9  not in train data dictionary
62292@N40  not in train data dictionary
62292@N40  not in train data dictionary
26495@N54  not in train data dictionary
1797@N28  not in train data dictionary
33638@N7  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
28961@N80  not in train data dictionary
20186@N94  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
29006@N14  not in train data dictionary
29006@N14  not in train data dictionary
29006@N14  not in train data dictionary
65630@N16  not in train data dictionary
38164@N13  not in train data dictionary
38164@N13  not in train data dictionary
38164@N13  not in train data dictionary
38164@N13  not in train data dictionary
38164@N13  n

99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary


14913@N9  not in train data dictionary
43650@N63  not in train data dictionary
64371@N34  not in train data dictionary
64371@N34  not in train data dictionary
46228@N11  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
1797@N28  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
99687@N61  not in train data dictionary
25257@N98  not in train data dictionary
25257@N98  not in train data dictionary
25257@N98  not in train data dictionary
2525

In [81]:
subcat_dict = total_train_json['Subcategory'].value_counts().to_dict()
train_subcontribution = []
test_subcontribution = []

for i in range(len(total_train_json)):
    Uid = total_train_json.iloc[i]['Uid']
    Subcategory = total_train_json.iloc[i]['Subcategory']
    uid_data = total_train_json[total_train_json["Uid"] == Uid]
    train_subcontribution.append(uid_data[uid_data['Subcategory'] == Subcategory].shape[0] / subcat_dict[Subcategory])

for i in range(len(test_json)):
    Uid = test_json.iloc[i]['Uid']
    Subcategory = test_json.iloc[i]['Subcategory']
    uid_data = test_json[test_json["Uid"] == Uid]
    if Subcategory not in subcat_dict:
        print("OHhhhhh NOooooooooo")
    test_subcontribution.append(uid_data[uid_data['Subcategory'] == Subcategory].shape[0] / subcat_dict[Subcategory])


In [85]:
train_features['uid_frequency'] = train_uid_frequency
test_features['uid_frequency'] = test_uid_frequency

train_features['subcat_contribute'] = train_subcontribution
test_features['subcat_contribute'] = test_subcontribution

In [86]:
train_features

,imgcat_avg,imgcon_avg,imgsubcat_avg,cat_encoded,year_encoded,month_encoded,day_encoded,hour_encoded,numberoftags,tag_weighted_value,uid_frequency,subcat_contribute
0,0.970739,0.948479,0.910685,6.401465,6.393873,6.582786,6.194953,6.346362,29,7.240832,4,0.002320
1,0.921197,0.912570,0.872287,6.187497,6.393873,6.531079,6.617993,6.269437,13,4.687871,160,0.565371
2,0.939743,0.933112,0.896570,6.176899,6.439919,6.605507,6.433589,6.071093,16,4.784232,160,0.565371
3,0.917213,0.909116,0.876409,6.176899,6.438576,6.588295,6.353670,6.405271,16,4.784232,160,0.565371
4,0.949377,0.941823,0.895207,6.187497,6.439919,6.531079,6.431376,5.933680,16,4.784232,160,0.565371
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.940390,0.906367,0.925911,6.118086,6.393873,6.232031,6.893098,6.242481,12,5.886721,3,0.004539
14996,0.970080,0.954597,0.804272,6.189877,6.396305,6.403020,6.893098,6.389846,31,8.385064,2,0.012821
14997,0.929996,0.771374,0.940127,6.203172,6.439919,6.243153,6.393108,6.242481,20,7.304288,1,0.002146
14998,0.895254,0.914376,0.765191,6.141163,6.438576,6.105631,5.822325,6.433724,60,7.270717,2,0.004175


In [87]:
train_features.to_csv("train_features.csv", index = False)
test_features.to_csv("test_features.csv", index = False)